# Import Library

In [1]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    TrainingArguments,
)
from accelerate import Accelerator

from trl import SFTConfig, SFTTrainer
from datasets import load_dataset, Dataset
import pickle
import wandb

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate 
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from peft import LoraConfig, get_peft_model

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# CONFIG

In [10]:
class CFG:
    # Root of Files
    BASE_DIRECTORY = './'
    DACON_TRAIN_CSV = './train.csv'
    DACON_TEST_CSV = './test.csv'

    EMBEDDINGS_ABOUT_TRAINING_DATASET = 'databases_multilingual-e5-base_training.pickle'
    EMBEDDINGS_ABOUT_TEST_DATASET = 'databases_multilingual-e5-base_test.pickle'

    EMBEDDING_CSV_FULL_TRAIN = './datas/stf_e5_base_full_train.csv'
    EMBEDDING_CSV_TRAIN = './datas/stf_e5_base_train.csv'
    EMBEDDING_CSV_VAL = './datas/stf_e5_base_val.csv'
    EMBEDDING_CSV_TEST = './datas/stf_e5_base_test.csv'

    # About Embedding
    EMBEDDING_MODEL = "intfloat/multilingual-e5-base"
    CHUNK_SIZE = 800
    CHUNK_OVERLAP = 50
    
    SEARCH_TYPE = "mmr"
    SEARCH_KWARGS_K = 3
    SEARCH_KWARGS_FETCH_K = 8

    # About Finetuning
    PRETRAINING_MODEL = "rtzr/ko-gemma-2-9b-it"

    LoRA_RANK = 16
    LoRA_ALPHA = 32
    LoRA_DROPOUT = 0.05

    TRAINING_RESUTL_DIR = './finetune_models/training_result'
    PER_TRAIN_BATCH_SIZE =2
    PER_EVAL_BATCH_SIZE = 2
    NUM_TRAIN_EPOCHS = 5
    LOGGING_DIR='./finetune_models/training_logs'
    LOGGING_STEPS=1000
    SAVE_STEPS=1000
    SAVING_FINETUNING_MODEL_DIR = "./finetune_models/gemma_ko_9b_ver1.01"

    PIPELINE_TEMPERATURE=0.2
    SEQ_MAX_LENGHT=450

    # About Submission
    SAMPLE_SUBMISSION = './sample_submission.csv'
    RESULT_SUBMISSION = './gemma_9b_ver1.02_submission.csv'


# DATA PREPARING 

### Vector DB

In [4]:
def process_pdf(file_path, chunk_size=CFG.CHUNK_SIZE, chunk_overlap=CFG.CHUNK_OVERLAP):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path=CFG.EMBEDDING_MODEL):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type=CFG.SEARCH_TYPE, 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

### TRAINING에 관한 EMBEDDING DB 생성

In [4]:
base_directory = CFG.BASE_DIRECTORY # Your Base Directory
train_df = pd.read_csv(CFG.DACON_TRAIN_CSV)
train_pdf_databases = process_pdfs_from_dataframe(train_df, base_directory)
pickle_file_path = os.path.join(base_directory, CFG.EMBEDDINGS_ABOUT_TRAINING_DATASET)

with open(pickle_file_path, 'wb') as f:
    pickle.dump(train_pdf_databases, f)

Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권...


c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:   6%|▋         | 1/16 [00:07<01:46,  7.09s/it]

Processing 2024 나라살림 예산개요...


Processing PDFs:  12%|█▎        | 2/16 [00:14<01:39,  7.12s/it]

...

Processing PDFs:  88%|████████▊ | 14/16 [02:14<00:39, 19.89s/it]

Processing 「FIS 이슈 & 포커스」 22-3호 《재정융자사업》...


Processing PDFs:  94%|█████████▍| 15/16 [02:30<00:18, 18.86s/it]

Processing 월간 나라재정 2023년 12월호...


Processing PDFs: 100%|██████████| 16/16 [03:33<00:00, 13.33s/it]


### TEST에 관한 EMBEDDING DB DB 생성

In [5]:
base_directory = CFG.BASE_DIRECTORY # Your Base Directory
test_df = pd.read_csv(CFG.DACON_TEST_CSV)
test_pdf_databases = process_pdfs_from_dataframe(test_df, base_directory)
pickle_file_path = os.path.join(base_directory, CFG.EMBEDDINGS_ABOUT_TEST_DATASET)

with open(pickle_file_path, 'wb') as f:
    pickle.dump(test_pdf_databases, f)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:04<00:35,  4.40s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:08<00:29,  4.25s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:28<00:08,  4.01s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:31<00:03,  3.79s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:35<00:00,  3.93s/it]


### EMBEDDING 정보를 저장한 CSV 파일의 생성을 위한 FUNCTIONS

In [5]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

def extract_context(df, pdf_databases, data_type='train'):
    # 결과를 저장할 리스트 초기화
    results = []

    # 배치 사이즈 설정
    batch_size = 1  # 원하는 배치 크기로 설정

    # DataFrame의 각 행에 대해 처리
    for start in tqdm(range(0, len(df), batch_size), desc="Creating Q&A including RAG info"):
        # 현재 배치 선택
        batch_rows = df.iloc[start:start + batch_size]

        # 배치 내의 각 행 처리
        for _, row in batch_rows.iterrows():
            # 소스 문자열 정규화
            source = normalize_string(row['Source'])
            question = normalize_string(row['Question'])
            if data_type == 'train':
                answer = normalize_string(row['Answer'])

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
            retriever = normalized_keys[source]['retriever']

            context = format_docs(retriever.invoke(question))

            if data_type == 'train':
                results.append({
                    "Context" : context,
                    "Question": question,
                    "Answer"  : answer,
                })
            else:
                results.append({
                    "Context" : context,
                    "Question": question,
                })

    return results


### TRAINING에 관한 CSV 파일 생성

In [4]:
train_df = pd.read_csv(CFG.DACON_TRAIN_CSV)
with open(CFG.EMBEDDINGS_ABOUT_TRAINING_DATASET, 'rb') as f:
    train_pdf_databases = pickle.load(f)

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [5]:
train_context = extract_context(train_df, train_pdf_databases, data_type='train')

stf_full_train_df = pd.DataFrame(train_context)
stf_train_df = pd.DataFrame(train_context[:int(len(train_context)*0.8+0.5)])
stf_eval_df = pd.DataFrame(train_context[int(len(train_context)*0.2+0.5):])

stf_full_train_df.to_csv(CFG.EMBEDDING_CSV_FULL_TRAIN, index=False, encoding="UTF-8")
stf_train_df.to_csv(CFG.EMBEDDING_CSV_TRAIN, index=False, encoding="UTF-8")
stf_eval_df.to_csv(CFG.EMBEDDING_CSV_VAL, index=False, encoding="UTF-8")

Creating Q&A including RAG info: 100%|██████████| 496/496 [00:17<00:00, 28.47it/s]


### TEST에 관한 CSV 파일 생성

In [3]:
test_df = pd.read_csv(CFG.DACON_TEST_CSV)
with open(CFG.EMBEDDINGS_ABOUT_TEST_DATASET, 'rb') as f:
    test_pdf_databases = pickle.load(f)

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
test_context = extract_context(test_df, test_pdf_databases, data_type='test')

stf_test_df = pd.DataFrame(test_context)

stf_test_df.to_csv(CFG.EMBEDDING_CSV_TEST, index=False, encoding="UTF-8")

Creating Q&A including RAG info: 100%|██████████| 98/98 [00:03<00:00, 28.96it/s]


# MODELING

### Loading default Pre-trained Model

In [3]:
# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 ID 
model_id = CFG.PRETRAINING_MODEL

# 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

# 모델 로드 및 양자화 설정 적용
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 10/10 [00:13<00:00,  1.35s/it]


### Show how the model look like

In [4]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

model.embed_tokens.weight True
model.layers.0.self_attn.q_proj.weight False
model.layers.0.self_attn.k_proj.weight False
model.layers.0.self_attn.v_proj.weight False
model.layers.0.self_attn.o_proj.weight False
model.layers.0.mlp.gate_proj.weight False
model.layers.0.mlp.up_proj.weight False
model.layers.0.mlp.down_proj.weight False
model.layers.0.input_layernorm.weight True
model.layers.0.post_attention_layernorm.weight True
model.layers.0.pre_feedforward_layernorm.weight True
model.layers.0.post_feedforward_layernorm.weight True
model.layers.1.self_attn.q_proj.weight False
model.layers.1.self_attn.k_proj.weight False
model.layers.1.self_attn.v_proj.weight False
model.layers.1.self_attn.o_proj.weight False
model.layers.1.mlp.gate_proj.weight False
model.layers.1.mlp.up_proj.weight False
model.layers.1.mlp.down_proj.weight False
model.layers.1.input_layernorm.weight True
model.layers.1.post_attention_layernorm.weight True
model.layers.1.pre_feedforward_layernorm.weight True
model.layer

### Preparing Parammeters for Finetuning

In [7]:
# Load LoRA configuration
peft_config = LoraConfig(
    r=CFG.LoRA_RANK,
    lora_alpha=CFG.LoRA_ALPHA,
    lora_dropout=CFG.LoRA_DROPOUT,
    bias="none",
    target_modules=[
    "model.embed_tokens", # able
    #"model.layers.0.input_layernorm", # unable
    #"model.layers.0.post_attention_layernorm", # unable
    #"model.layers.0.pre_feedforward_layernorm", # unable
    #"model.layers.0.post_feedforward_layernorm", # unable
    #"model.norm" # unable
    ],
    task_type="CAUSAL_LM",
)        

training_args = TrainingArguments(
    output_dir=CFG.TRAINING_RESUTL_DIR,
    per_device_train_batch_size=CFG.PER_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=CFG.PER_EVAL_BATCH_SIZE,
    num_train_epochs=CFG.NUM_TRAIN_EPOCHS,
    logging_dir=CFG.LOGGING_DIR,
    logging_steps=CFG.LOGGING_STEPS,
    save_steps=CFG.SAVE_STEPS,
    evaluation_strategy="steps",
)

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Finetuning and Saving Finetuned Model

In [8]:
def Finetune_llm_with_SFT_Trainer():

    def normalize_string(s):
        """유니코드 정규화"""
        return unicodedata.normalize('NFC', s)
    
    # load dataset
    # train_dataset = load_dataset('csv', data_files=CFG.EMBEDDING_CSV_TRAIN)['train']
    # eval_dataset = load_dataset('csv', data_files=CFG.EMBEDDING_CSV_VAL)['train']  
    train_dataset = load_dataset('csv', data_files=CFG.EMBEDDING_CSV_FULL_TRAIN)['train']
    
    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example)):
            text =  """다음 정보를 바탕으로 질문에 답하세요. 답변은 꼭 문장으로 하세요. 주어를 꼭 적으세요. :
# {example[Context]}
# 
# 질문: {example[Question]}
# 
# 답변: {example[Answer]}
# """
            text = normalize_string(text)
            output_texts.append(text)
        return output_texts
    
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        peft_config = peft_config,
        formatting_func=formatting_prompts_func,
        train_dataset = train_dataset,
        # eval_dataset = eval_dataset,   
    )

    # Train model
    trainer.train()
    
    # Save trained model
    trainer.model.save_pretrained(CFG.SAVING_FINETUNING_MODEL_DIR)
    tokenizer.save_pretrained(CFG.SAVING_FINETUNING_MODEL_DIR)

In [9]:
Finetune_llm_with_SFT_Trainer()

Generating train split: 496 examples [00:00, 9350.04 examples/s]
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\trl\trainer\sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 496/496 [00:00<00:00, 24777.28 examples/s]
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\trl\trainer\sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING Th

  0%|          | 0/10 [00:00<?, ?it/s]It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\models\gemma2\modeling_gemma2.py:458: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 10/10 [00:07<00:00,  1.43it/s]


{'train_runtime': 10.0235, 'train_samples_per_second': 1.496, 'train_steps_per_second': 0.998, 'train_loss': 2.4764156341552734, 'epoch': 5.0}


### Loading Finetuned Model (for Quantization) and Make Pipeline

In [5]:
def setup_llm_SFTTrainer_with_finetuning():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    # loading model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(CFG.SAVING_FINETUNING_MODEL_DIR, 
                                                 quantization_config=bnb_config,
                                                 trust_remote_code=True,
                                                 device_map="auto",)
    tokenizer = AutoTokenizer.from_pretrained(CFG.SAVING_FINETUNING_MODEL_DIR)

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=CFG.PIPELINE_TEMPERATURE,
        return_full_text=False,
        max_new_tokens=CFG.SEQ_MAX_LENGHT, 
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [6]:
llm = setup_llm_SFTTrainer_with_finetuning()

Loading checkpoint shards: 100%|██████████| 10/10 [00:07<00:00,  1.26it/s]
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


# EVALUATION

### Utils for Evaluation

In [7]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

### evaluation

In [8]:
# 결과를 저장할 리스트 초기화
results = []

# CSV 파일 읽기
df = pd.read_csv(CFG.EMBEDDING_CSV_TEST)

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    context = normalize_string(row['Context'])
    question = normalize_string(row['Question'])

    # RAG 체인 구성
    template = """다음 정보를 바탕으로 질문에 답하세요. 답변은 꼭 문장으로 하세요. 주어를 꼭 적으세요. :
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = ( 
        prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    # print(f"Question: {question}")
    full_response = rag_chain.invoke({"context":context, "question":question})

    # print(f"Answer: {full_response}\n")
    
    # 결과 저장
    results.append({
        'Question': question,
        'Context': context,
        'Answer': full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\models\gemma2\modeling_gemma2.py:458: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Answering Questions:   1%|          | 1/98 [00:06<09:42,  6.00s/it]c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set t

# SUBMISSIOIN

In [11]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(CFG.SAMPLE_SUBMISSION)

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv(CFG.RESULT_SUBMISSION, encoding='UTF-8-sig', index=False)